
<center>
    <h1><a href="http://diplodatos.famaf.unc.edu.ar/">Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</a></h1>
    <h2>Curso <a href="https://sites.google.com/view/eleccion-optativas-diplodatos/programaci%C3%B3n-distribu%C3%ADda-sobre-grandes-vol%C3%BAmenes-de-datos">Programación Distribuida sobre Grandes Volúmenes de Datos</a></h2>
</center>

<br>

<h3 style="text-align:center;"> Damián Barsotti  </h3>

<h3 style="text-align:center;">
    <a href="http://www.famaf.unc.edu.ar">
    Facultad de Matemática Astronomía Física y Computación
    </a>
<br/>
    <a href="http://www.unc.edu.ar">
    Universidad Nacional de Córdoba
    </a>
<br/>
    <center>
    <a href="http://www.famaf.unc.edu.ar">
    <img src="https://www.famaf.unc.edu.ar/static/assets/logosFooterBottom.svg" alt="Drawing" style="width:50%;"/>
    </a>
    </center>
</h3>

<p style="font-size:15px;">
    <br />
        This work is licensed under a
        <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.
    <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
        <img alt="Creative Commons License" style="border-width:0;vertical-align:middle;float:right" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
    </a>
</p>


# Introducción a Spark
---

## Características

* 100x más rápido que Hadoop MapReduce en memoria.
* 10x más rápido en disco.

  ![](https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/HEAD/clases/01_intro_spark/spark_speed.png)
  
* Multiplataforma
    * Corre en Hadoop Yarn, Mesos, standalone o en la nube (AWS, Azure, ...)
    * Acceso a datos en HDFS, Cassandra, HBase, Hive, Tachyon, JDBC, etc.
![](https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/HEAD/clases/01_intro_spark/spark_multi_plataforma.png)

## Múltiples funcionalidades en una plataforma (Stack unificado)

<img src="https://bitbucket.org/bigdata_famaf/diplodatos_bigdata/raw/b17129f7118b3389b8c7f2f85fd89c6238fe0edd/clases/01_intro_spark/unified_stack.png" alt="Drawing" style="width: 80%;"/>

## Word Count (MapReduce)

```java
public class WordCount {
	public static class Map extends MapReduceBase implements Mapper<LongWritable, Text, Text, IntWritable> {

		private final static IntWritable one = new IntWritable(1);
		private Text word = new Text();
        
		public void map(LongWritable key, Text value, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {

			String line = value.toString();
			StringTokenizer tokenizer = new StringTokenizer(line);

			while (tokenizer.hasMoreTokens()) {
				word.set(tokenizer.nextToken());
				output.collect(word, one);
			}
		}
	}

	public static class Reduce extends MapReduceBase implements Reducer<Text, IntWritable, Text, IntWritable> {

		public void reduce(Text key, Iterator values, OutputCollector<Text, IntWritable> output, Reporter reporter) throws IOException {

			int sum = 0;
			while (values.hasNext()) {
				sum += values.next().get();
			}
			output.collect(key, new IntWritable(sum));
		}
	}

	public static void main(String[] args) throws Exception {

		JobConf conf = new JobConf(WordCount.class);
		conf.setJobName("wordcount");
		conf.setOutputKeyClass(Text.class);
		conf.setOutputValueClass(IntWritable.class);
		conf.setMapperClass(Map.class);
		conf.setCombinerClass(Reduce.class);
		conf.setReducerClass(Reduce.class);
		conf.setInputFormat(TextInputFormat.class);
		conf.setOutputFormat(TextOutputFormat.class);
		FileInputFormat.setInputPaths(conf, new Path(args[0]));
		FileOutputFormat.setOutputPath(conf, new Path(args[1]));
		JobClient.runJob(conf);
	}
}
```

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("01_intro").getOrCreate()
sc = spark.sparkContext

## Word Count (Spark)

* `lines` es un **array distribuido** de lineas de texto (`RDD[str]`).
    - una parte del arreglo en cada **nodo del cluster**.

* `lines` tiene el método `flatMap` (línea 6):
    - `flatMap(lambda line: line.split(" "))` toma cada cada elemento del `RDD` (linea), lo convierte en sequencia de palabras y concatena estas secuencias:
        - `lambda line: line.split(" ")` es la **función** que toma una linea y la divide en una secuencia de palabras.
        
    - Su resultado es un array **distribuido** de palabras (`RDD[str]`).
    
* Al resultado de `flatMap` se aplica el método `filter` (línea 7):
    - `filter(lambda word: word)` saca las palabras que son vacías (pueden aparecer?).
    - `lambda word: word` es la **función** que pregunta si la palabra es vacía.
    - `filter` devuelve un `RDD` que se almacena en `words`.

* `words` tiene el método `map` (línea 11):
    - `map(lambda word: (word,1))` agrega a cada palabra de `words` un `1`.
    - El resultado es un **arreglo distribuido** de tuplas `RDD[(str, Int)]`.
    
* A este `RDD` se le aplica el método `reduceByKey` (línea 12):
    - `reduceByKey(lambda n,m: n+m)` suma los `1`'s de las palabras iguales (la key es la palabra).


In [3]:
lines = sc.textFile("../inputs/README.md")

words = lines \
    .flatMap(lambda line: line.split(" ")) \
    .filter(lambda word: word)

## filter-devuele true si no es vacia. 
#MapReduce
wordCount = words \
    .map(lambda word: (word,1)) \
    .reduceByKey(lambda n,m: n+m)

## Resultado Word Count Spark

In [4]:
result = wordCount.sortBy((lambda p: p[1]), ascending = False) # ordena por cantidad

local_result = result.collect() # Traigo desde cluster

for word, count in local_result[:10]: # tomo 10
    print(word, count) # los imprimo

from 4
Apache 3
Zeppelin 3
and 3
to 3
* 2
### 2
binary 2
Please 2
[User 2


## RDD de entrada

In [5]:
inputRDD = sc.textFile("../inputs/logs/zeppelin-interpreter-spark-*-nabucodonosor.log") 
# se crea un nuevo RDD:
skarkRDD = inputRDD.filter(lambda line: "Successfully started service 'SparkUI' on port " in line) 

for ln, l in enumerate(skarkRDD.collect()):
    print("Linea {}:".format(ln), l)

Linea 0:  INFO [2020-10-31 10:15:43,582] ({pool-2-thread-3} Logging.scala[logInfo]:54) - Successfully started service 'SparkUI' on port 4044.


## Ejercicio 0 (word count)
* Crear una celda abajo de esta (poner mouse debajo de esta celda y seleccionar "Add Paragraph").
* Copiar el programa `wordcount` anterior en la misma (esta en 2 celdas).
    - [`shift`]-[`flechas`] para seleccionar.
    - [`ctrl`]-[`c`] para copiar.
    - [`ctrl`]-[`v`] para pegar.
* Modificarlo para leer todas la lineas de los archivos en `./licenses/`
    - Ayuda: si al método `textFile` se le indica el nombre de un directorio carga todos los archivo del mismo.
* Ejecute la celda ([`shift`]-[`enter`])
* Ver la cantidad de tareas en SparkUI

In [6]:
lines = sc.textFile("../inputs/licenses/")

words = lines \
    .flatMap(lambda line: line.split(" ")) \
    .filter(lambda word: word) 
    
## filter-devuele true si no es vacia. 

#MapReduce
wordCount = words \
    .map(lambda word: (word,1)) \
    .reduceByKey(lambda n,m: n+m)

local_result = wordCount.collect() # Traigo desde cluster

for word, count in local_result[:10]: # tomo 10
    print(word, count) # los imprimo

ARE 17
SERVICES; 17
common 7
(ii) 7
code, 10
"Not 5
Subject 10
publicly 10
perform, 7
litigation 12


Ver la cantidad de tareas en SparkUI - Rta: 
 * Si no se ejecuta la instruccion `collect()` No se ejecuta ninguna tarea
   Estp es por que spark tiene el enofoque lazy, no evalua nada y no ejecuta nada
   hasta se necesite el re
 * si se ejecuta, Utiliza 124/124 tareas 

## Ejecución de programas en Spark

* En [Zeppelin](http://zeppelin.apache.org/) (como lo hacemos ahora)
* En `pyspark` shell (tambien interactivo)
* Como programa autónomo

### pyspark shell

* Abrir una terminal
* Ir a la instalación Spark
```sh
cd ~/spark/spark-2.3.4-bin-hadoop2.7
```
* Arrancar el shell
```sh
./bin/pyspark
```
* Escribir en shell (después apretar `Enter`)
```python
>>> lines = sc.textFile("README.md")
>>> lines.first()
```

### Programa autónomo

* Ir a programa
```sh
cd diplodatos_bigdata/prog/word_count
```
* Actualizar repositorio
```sh
git pull
```
* Ver programa
```sh
less src/main/python/WordCount.py
```
  (salir con [`q`])
  
### Ejecucion de programa

* Ejecutar
```sh
~/spark/spark-2.3.4-bin-hadoop2.7/bin/spark-submit --master local[4] src/main/python/WordCount.py \
        ~/spark/spark-2.3.4-bin-hadoop2.7/licenses/
```

### Versión Spark en Zeppelin

In [7]:
print(sc.version)

3.0.1


### Principales referencias online:

* [Documentación Spark](http://spark.apache.org/docs/2.2.1/)
* [API Spark Python](http://spark.apache.org/docs/2.2.1/api/python/index.html)

## Ejercicios MapReduce con Spark
---

### Ejercicio 1

Modifique el programa *word count* siguiente para que cuente la **cantidad de apariciones de cada letra** en el archivo.

* Ayuda: solo hay que modificar la linea 6

In [8]:
lines = sc.textFile("../inputs/README.md")

words = lines \
    .flatMap(lambda line: list(line)) \
    .filter(lambda word: word != ' ')


#MapReduce
wordCount = words \
    .map(lambda word: (word,1)) \
    .reduceByKey(lambda n,m: n+m)

result = wordCount \
    .sortBy((lambda p: p[1]), ascending=False) # ordena por cantidad

local_result = result.collect() # Traigo desde cluster

for word, count in local_result[:10]: # tomo 10
    print(word, count) # los imprimo

e 102
t 92
a 83
i 73
p 61
o 59
n 58
s 56
l 54
r 54


### Ejercicio 2

Cada línea del archivo `~/diplodatos_bigdata/ds/links_raw.txt` contiene un url de una página web seguido de los links que posee a otras páginas web:
```
<url> <url link 1> <url link 2> ... <url link n>
```

Basándose en la utilización de la técnica de *MapReduce* que se mostró en el programa `word count` haga un programa en Spark que cuente la cantidad de links que apuntan a cada página.

#### Ayuda

A continuación está el comienzo del programa. Falta hacer el *MapReduce* y mostrar el resultado.


In [9]:
baseDir = "../inputs" # llenar con el directorio git

lines = sc.textFile(baseDir + "/ds/links_raw.txt")

linksTo = lines \
    .flatMap(lambda l: l.split(" ")[1:]) # separo los links y tomo los apuntados

# Ahora linksTo tiene las paginas apuntadas

# Completar los ...

# MapReduce
invLinkCount = linksTo.map(lambda link: (link,1)) \
    .reduceByKey(lambda n,m: n+m)

result = invLinkCount.sortBy((lambda p: p[1]), ascending = False)

local_result = result.collect() # Traigo desde cluster

for word, count in local_result[:10]: # tomo 10
    print(word, count) # los imprimo

http://www.yahoo.com/ 199
http://www.ca.gov/ 169
http://www.leginfo.ca.gov/calaw.html 155
http://www.linkexchange.com/ 134
http://www.berkeley.edu/ 126
http://www.sen.ca.gov/ 123
http://home.netscape.com/comprod/mirror/index.html 109
http://www.assembly.ca.gov/ 99
http://www.epa.gov/ 95
http://www.usgs.gov/ 84


FIN